In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import seaborn as sns
import sqlite3
import glob
from matplotlib import pyplot as plt

/Users/hari/.pyenv/versions/3.6.5/envs/covid/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
path = r'/Users/hari/python/py3/covid/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports' 
all_files = glob.glob(path + "/*.csv")
print(len(all_files))

79


In [5]:
!rm covid.db
conn = sqlite3.connect("covid.db")
for filename in all_files:
    df=pd.read_csv(filename,header=0,index_col=None)
    df=df.rename(columns={"Country/Region": "Country", 
                          "Country_Region":"Country",
                          "Province/State": "Province",
                          "Province_State":"Province",
                          "Last_Update":"LastUpdate",
                          "Last Update":"LastUpdate",
                          "Lat":"Latitude",
                          "Long_":"Longitude",
                          "Combined_Key":"CombinedKey"})
    if not "Latitude" in df.columns:
        df['Latitude']=None
    if not "Longitude" in df.columns:
        df['Longitude']=None
    if not "Active" in df.columns:
        df['Active']=None
    if not "CombinedKey" in df.columns:
        df['CombinedKey']=None


    if 'FIPS' in df.columns:
        df = df.drop("FIPS",axis=1)
    if 'Admin2' in df.columns:
        df = df.drop("Admin2",axis=1)
    df['Date']=pd.to_datetime(df['LastUpdate']).dt.date
    df_country=df[['Date','Country','Confirmed','Deaths','Recovered']].groupby(['Date','Country']).sum()
    df_country.to_sql('daily',conn,if_exists='append')
    df_latlon=df[['Date','Country','Latitude','Longitude','Confirmed','Deaths','Recovered']].groupby(['Date','Country','Latitude','Longitude']).sum()
    df_latlon.to_sql('daily_latlon',conn,if_exists='append')

conn.execute("update daily set Country='South Korea' where Country like 'Korea, South'")
conn.execute("update daily set Country='South Korea' where Country like 'Republic Of Korea'")
conn.execute("update daily_latlon set Country='South Korea' where Country like 'Korea, South'")
conn.execute("update daily_latlon set Country='South Korea' where Country like 'Republic Of Korea'")

conn.commit()
conn.close()